## Nextgenmap
Runs `nextgenmap` in paired and single end setting. Samples are grouped by `R1` and `R2` in file names.
If the keyword `R` is not found its treated as a single ended file.


In [ ]:
import logging
import os
import subprocess
import sys
import traceback
import stat

from hops import hdfs
from pyspark import SparkContext

import utils


#### Load arguments

In [ ]:


args=utils.load_arguments(sys.argv)
#args=utils.load_arguments([0,'hdfs:///Projects/TCGA_viruses/Jupyter/pipeline/settings_DJ.yml'])

if args is not None:
    args=args[utils.KEY_NGM]
else :
    sys.exit(utils.NO_CONFIG_ERR)
    
sc = SparkContext.getOrCreate()

VERY_FAST='--very-fast'
INPUT_ROOT=args['INPUT_ROOT']
OUTPUT_ROOT=args['OUTPUT_ROOT']
REFERENCE_PATH=args['REFERENCE_FILE']
MIN_I=str(args['MIN-IDENTITY'])
MIN_R=str(args['MIN-RESIDUES'])
THREADS=args['THREADS']
LOG_DIR=args['LOGS_ROOT']
is_very_fast=args['VERY_FAST']
SPACE=utils.SPACE
TOOL_PATH='hdfs:///Projects/4C/Tools/ngm_built/NGM'
TOOL='./NGM/bin/ngm-0.5.5/ngm'



def chmod_exec(tool):
    st = os.stat(tool)
    os.chmod(tool, st.st_mode | stat.S_IEXEC)

def install_ngm(tool_path):
    hdfs.copy_to_local(tool_path)
    lib1='./NGM/bin/ngm-0.5.5/ngm-core'
    lib2='./NGM/bin/ngm-0.5.5/ngm'
    chmod_exec(lib1)
    chmod_exec(lib2)

#### Map functions 

In [ ]:
"""
Runs NGM in paired mode via subprocess for single R1 R2 pair.
Output is saved in SAM format and copied back to hdfs.
"""
def apply_ngm_paired(x,REFERENCE_PATH): ### paired files

    filename_forward_path=x[0] #r1
    filename_reverse_path=x[1] #r2
    # split path to get file names
    filename_forward=os.path.basename(filename_forward_path)
    filename_reverse=os.path.basename(filename_reverse_path)

    output_file=filename_forward.split('.')[0].replace(utils.R1,'')
    if  utils.skip_file(filename_forward,output_file,OUTPUT_ROOT):
        return [-1]

    if not os.path.exists(os.path.basename(TOOL_PATH)):
        install_ngm(TOOL_PATH)

    # get file name and copy to local
    ref=os.path.split(REFERENCE_PATH)[1]
    if not os.path.exists(ref):        
        hdfs.copy_to_local(REFERENCE_PATH)
    hdfs.copy_to_local(filename_forward_path, overwrite=True)
    hdfs.copy_to_local(filename_reverse_path, overwrite=True)
    

    parameters = { '-i':MIN_I, '-R': MIN_R, '-p': utils.EMPTY, '-r': ref, '-1': filename_forward, '-2': filename_reverse,
                  '--silent-clip': utils.EMPTY, '-o': output_file, '-t': THREADS, '--no-progress': utils.EMPTY }
    
    cmd = utils.build_command(TOOL,parameters)
    if is_very_fast :
        cmd=cmd+SPACE+VERY_FAST # final command to run
    
    logging.info('Running nextgenmap with command:', cmd)
    log_file=os.path.splitext(output_file) [0]+'.txt'
    f=open(log_file, "w")
    # run
    try:
        execStatus=subprocess.run(cmd.split(),stdout=f,stderr=f)
        hdfs.copy_to_hdfs(log_file, LOG_DIR, overwrite=True)
        f.close()
        status=False
        if execStatus.returncode==0 and os.path.exists(output_file):
             # copy output to hdfs
            hdfs.copy_to_hdfs(output_file, OUTPUT_ROOT, overwrite=True)
            status=True
            # remove from local
            os.remove(output_file)
    except Exception as e:
        traceback.print_exc()
        raise RuntimeError('Failed execution for input ', x )
    finally:
        os.remove(filename_forward)
        os.remove(filename_reverse)
        os.remove(log_file)

        parameters.clear()
    
    return [status,output_file]


"""
Runs NGM in single mode for single file.
Output is in SAM format and copied back to hdfs.
"""
def apply_ngm_single(x,REFERENCE_PATH): ### paired files
    
    
    filename_forward_path=x
    # split path to get file names
    filename_forward=os.path.basename(filename_forward_path)
    output_file=filename_forward.split('.')[0].replace(utils.R1,'')+'.sam'
    # skip if output exists
    if  utils.skip_file(filename_forward,output_file,OUTPUT_ROOT):
         return [-1]
    # get file name
    ref=os.path.split(REFERENCE_PATH)[1]
    if not os.path.exists(ref):
        hdfs.copy_to_local(REFERENCE_PATH,overwrite=False)

    hdfs.copy_to_local(filename_forward_path, overwrite=True)

    parameters = { '-i':MIN_I, '-R': MIN_R, '-q': filename_forward, '-r': ref, 
                  '--silent-clip': utils.EMPTY, '-o': output_file, '-t': THREADS, '--no-progress': utils.EMPTY }
    
    cmd = utils.build_command(TOOL,parameters)
    
    if is_very_fast :
        cmd=cmd+SPACE+VERY_FAST # final command to run
    
    logging.info('Running nextgenmap with command:', cmd)
    log_file=os.path.splitext(output_file) [0]+'.txt'
    f=open(log_file, "w")
    # run command    
    subprocess.run(cmd.split(),stdout=f,stderr=f)
    
    hdfs.copy_to_hdfs(log_file, LOG_DIR, overwrite=True)
    f.close()
    status=False
    if os.path.exists(output_file):    
        # copy to hdfs
        hdfs.copy_to_hdfs(output_file, OUTPUT_ROOT, overwrite=True)
        status=True
        # remove from local 
        os.remove(filename_forward)    
        os.remove(output_file)
        os.remove(log_file)
        
    parameters.clear()          
    
    return [status,output_file]
            

#### List all input files hdfs path

In [ ]:


all_files=utils.load_file_names(INPUT_ROOT)

#### Get list of all single end files and run NGM in single mode in parallel

In [ ]:
### single
single_files=[f for f in all_files if utils.R_IDENTIFIER not in f]


print('number of single input  files processing ', len(single_files))
dataRdd=sc.parallelize(single_files)

# run
ngmSingleFiles=dataRdd.map(lambda x: apply_ngm_single(x,REFERENCE_PATH)).collect()

#### Pair R1 and R2 as a tuple in a list and run NGM in paired end in parallel

In [ ]:

# group r1 and r2
pairedList = utils.group_R1R2(all_files)
dataPairedRdd=sc.parallelize(pairedList,sc.getConf().get("spark.executor.instances"))

# run
try:
    ngmFiles=dataPairedRdd.map(lambda x: apply_ngm_paired(x,REFERENCE_PATH) ).collect()
except Exception as e:
    traceback.print_exc()
    raise SystemError('Failed job execution')